In [3]:
import os
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import pandas as pd
import json

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

from tensorflow.keras.applications import MobileNet

from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix
import itertools

print(f"Tensor Flow Version: {tf.__version__}")
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Tensor Flow Version: 2.0.0-rc1
GPU is available
Num GPUs Available:  1


In [4]:
## Getting the directories for the train and validation data
base_dir = os.getcwd()
train_dir = os.path.join(base_dir, 'final/train/')
validation_dir = os.path.join(base_dir, 'final/val/')

train_dir = train_dir.replace(os.sep,'/')
validation_dir=validation_dir.replace(os.sep,'/')


print(train_dir)
print(validation_dir)


D:/Projects/dogSpecies/final/train/
D:/Projects/dogSpecies/final/val/


In [5]:
# Counting number of datapoints in both train and validation

total_train = 0
total_val = 0
total_test = 0

for x in next(os.walk(train_dir))[1]:
    for y in next(os.walk(os.path.join(train_dir,x)))[2]:
        #print(y)
        total_train=total_train+1
        
for x in next(os.walk(validation_dir))[1]:
    for y in next(os.walk(os.path.join(validation_dir,x)))[2]:
        #print(y)
        total_val=total_val+1


print(total_train)
print(total_val)


16418
4162


In [6]:
BATCH_SIZE = 10  
IMG_SHAPE  = 224  # new shape of the desired image

In [7]:
# Generating Train data

image_gen_train = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      horizontal_flip=True)

train_data = image_gen_train.flow_from_directory(batch_size=BATCH_SIZE,
                                                     directory=train_dir,
                                                     target_size=(IMG_SHAPE,IMG_SHAPE),
                                                     class_mode='binary')

Found 16418 images belonging to 120 classes.


In [8]:
# Generating Validation data

image_gen_val = ImageDataGenerator(rescale=1./255)

val_data = image_gen_val.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory=validation_dir,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='binary')

Found 4162 images belonging to 120 classes.


In [10]:
# Generating validation data labels and saving it to the disk as a txt file

df = pd.DataFrame(columns=['breed', 'class'])

val_label = val_data.classes
val_data.class_indices

with open('class_label.txt', 'w') as outfile:
    json.dump(val_data.class_indices, outfile)

In [11]:
# sanity check!! checking the bath size and dimensions of the training image

sample_training_images, _ = next(train_data)
sample_training_images.shape

(10, 224, 224, 3)

In [12]:
# downloading the MobileNet base model

base_model=MobileNet()

#Replacing the last six layers of the model and adding our own softmax layer with 120 number of classes
x = base_model.layers[-6].output 
predictions = tf.keras.layers.Dense(120,activation ='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs = predictions)

# printing the new model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128   

In [13]:
# setting last 11 layers as trainable
for layer in model.layers[:-11]:
    layer.trainable = False

In [14]:
# Compiling model with adam optimizer

from tensorflow.keras.optimizers import Adam

model.compile(Adam(lr=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
# Training the model with 20 epochs. 

EPOCHS = 20
history = model.fit_generator(
    train_data,
    steps_per_epoch=int(total_train / BATCH_SIZE),
    epochs=EPOCHS,
    validation_data=val_data,
    validation_steps=int(total_val / BATCH_SIZE)
)

Epoch 1/20


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]

In [ ]:
# Saving the weights and entire model to disk

model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
